In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

header_names=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
              'DIS', 'RAD', 'TAX', 'PTRATION', 'B', 'LSTAT', 'PRICE']
df = pd.read_csv('../input/boston-house-prices/housing.csv', names=header_names, delim_whitespace=True)
df.head()

In [ ]:
df1=df.copy()
df.info()

In [ ]:
df.describe()

Exploratory Data Analysis


In [ ]:
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.corr()

In [ ]:
sns.pairplot(df[['CRIM', 'ZN', 'INDUS', 'CHAS',  'PRICE']])

crime rate is low and price is high


In [ ]:
sns.pairplot(df[['NOX', 'RM', 'AGE','DIS','RAD','PRICE']])

* (nitric oxide valuee parts per million)nox increases price decreases
* rm(average number of rooms per dwelling) increases price increases
* age increases price decreases(lowest price is at highest age of house)
* DIS(weighted distance to 5 employment centers) also shows a relationship
* RAD less price more(accessible highways distance)that means when the distance to highways is less 

In [ ]:
   sns.pairplot(df[[ 'TAX', 'PTRATION', 'B', 'LSTAT','PRICE']])

*   Tax is less price is more
*  proportion of black is more and prices are higher
*  Lstat(lower status of population) shows a curve with price(to be explained in detail)

**identyfing numeric and categorial columns**

In [ ]:
def numeric_features(df):
    numeric_col = df.select_dtypes(include=np.number).columns.tolist()
    return df[numeric_col].head()
numeric_columns = numeric_features(df)
print("Numerical Features:")
print(numeric_columns)


In [ ]:
def categorical_features(dataset):
    categorical_col = dataset.select_dtypes(exclude=np.number).columns.tolist()
    return dataset[categorical_col].head()

categorical_columns = categorical_features(df)
print("Categorical Features:")
print(categorical_columns)

print("===="*20)


**Detecting outliers**

In [ ]:
def detect_outliers(df):
    cols = list(df)
    outliers = pd.DataFrame(columns = ['Feature', 'Number of Outliers'])
    for column in cols:
        if column in df.select_dtypes(include=np.number).columns:
            q1 = df[column].quantile(0.25)
            q3 = df[column].quantile(0.75)
            iqr = q3 - q1
            fence_low = q1 - (1.5*iqr)
            fence_high = q3 + (1.5*iqr)
            outliers = outliers.append({'Feature':column, 'Number of Outliers':df.loc[(df[column] < fence_low) | (df[column] > fence_high)].shape[0]},ignore_index=True)
    return outliers

detect_outliers(df)

using winsorzation to remove outliers

In [ ]:
from scipy.stats.mstats import winsorize

# Function to treat outliers 

def treat_outliers(dataframe):
    cols = list(dataframe)
    for col in cols:
        if col in dataframe.select_dtypes(include=np.number).columns:
            dataframe[col] = winsorize(dataframe[col], limits=[0.05, 0.1],inclusive=(True, True))
    
    return dataframe    


df = treat_outliers(df)

# Checking for outliers after applying winsorization
# We see this using a fuction called 'detect_outliers', defined above.

detect_outliers(df)

the columns still showing outliers are not outliers they are the actual values

dropping CHAS column as it shows less correlation and doesn't contribute to price prediction

In [ ]:
df.drop('CHAS',axis=1)

No**w training the model**

In [ ]:
!pip install scikit-learn --quiet

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
y=df['PRICE']
X=df.drop(['PRICE','CHAS'],axis=1,inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
model = LinearRegression().fit(X_train,y_train)

In [ ]:
model.coef_

In [ ]:
predictions=model.predict(X_test)

In [ ]:
sns.scatterplot(x=y_test,y=predictions)

checking accuracy

In [ ]:
model.score(X_test,y_test)

checking errors

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
print('MAE:', mean_absolute_error(y_test,predictions))
print('MSE:', mean_squared_error(y_test,predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test,predictions)))

In [ ]:
pd.DataFrame(model.coef_,X.columns, columns=['Coeffecient'])

* 1 unit of increase in crime rate(CRIM) results in decrease of price by -0.509557
* 1 unit increase in nitric oxide concentration(NOX) is associated with decrease in -21.554589
* 1 unit increase in RM(average number of rooms per dwelling) is associated with increase in 2.766017